# Brusselator

Sylvester equation can be used as a tool to solve matrixequations that arise in the spacial discretization of PDEs. PDEs model complex spatial-temporal phenomena like the patterns in Belouzov-Zhabotinky reactions, and their solutions can be found using matix-oriented finite element methods which lead to Sylvester-type equations.
1. If the reaction written as a stoichiometric matrix that lead to a dynamical system can be rewritten in the form $F'(X,Y)=A(X,Y)+C$, the Sylvester-like euqation can be derived from there.
2. A fist discretization using the reaction diffusion system form can also be studied for further different results.
3. Further work could entail time study of the oscillations process (for the Oregonator).

### Imports

In [ ]:
import numpy as np
import sympy as sp
import scipy as sc
from scipy import optimize
#from scipy.integrate import Radau #Ref. CAMP Polytechnique Paris
import pytensor
import pytensor.tensor as pt
#import pymepack as pme
#from pymepack import gesylv

### Using Symbolic values

The system to be defined is on the form $F'(X,Y)=A(X,Y)+C$ with a quadratic variable $X^2$

Define the lifting state as $V=(X,Y,X^2,XY)$

There are 6 possible decompositions such that $F'(V)=MV+C$ (under the conditions that are not fully verified yet)

First possible decomposition

In [23]:
# Variable concentrations
X = sp.Symbol('X')
Y = sp.Symbol('Y')

# Values for the kinetics and the constant concentrations
k1 = 1.28
k2 = 2.4*10**6
k3 = 33.6
k4 = 2400.0
A = 0.06
B = 0.02
D = 1.0
E = 1.0

M = np.array([[-k2*B-k4, k3*X**2, 0, 0], [k2*B, k3*X**2, 0, 0]])
C = np.array([[k1*A], [0]])

**Define the Sylvester equation**

Suppose two systems such that $F'(V1)=M1*V1+C1$ and $F'(V2)=M2*V2+C2$, there are again 6 possible decompositions

Define the linear mapping that should be obtained as $V2=V*V1$

Then $F'(V2)=VF'(V1)=V(M1*V1+C1)=V*M1*V1+V*C1$ and $F'(V2)=M2*V2+C2=M2*V*V1+C2$

Which gives $V*M1*V1+V*C1=M2*V*V1+C2 \Rightarrow V*M1-M2*V=C2-V*C1$ (Sylvester like equation)

The verification of the conditions for the standard and the general Sylverster equations solution are ommited

Rewrite the Sylvester like equation : $V*(M1+C1)-M2*V=C2 \Leftrightarrow -M2*V+V*(M1+C1)=C2 \Leftrightarrow NV+VP=C$

In [39]:
M1 = np.array([-k2*B-k4, k3*X**2, 0, 0])
C1 = k1*A
M2 = np.array([k2*B, -k3*X**2, 0, 0])
C2 = 0
N = -M2
P = M1+C1
R = N*np.array([X, Y, X**2, X*Y])+np.array([X, Y, X**2, X*Y])*P-C
#V, *_ = pme.gesylv(N, P, C) # solve the standard Sylvester equation
#res = pme.res_gesylv(N, P, V, C) # compute the residual

### Using an optimization solver

In [40]:
# The method is using symbolic values, which does not allow to use optimize root directly form SciPy (optimize.root) to find the residual of the Sylvester equation, whence the use of pytensor.
X, Y = pt.tensor('variables', shape=(2,))
A, B, D, E = pt.scalars('A B D E'.split())
k1, k2, k3, k4 = pt.scalars('k1 k2 k3 k4'.split())
eq1 = k1*A-k2*B*X+k3*X**2*Y-k4*X
eq2 = k2*B*X-k3*X**2*Y
fun = np.array([[k1*A-k2*B*X+k3*X**2*Y-k4*X], [k2*B*X-k3*X**2*Y]])
#sol, success = pt.optimize.root(fun, np.array([[X], [Y]])) #Find the roots of a multivariate function using MINPACK’s hybrd and hybrj routines (modified Powell method)

### Using a discretization method

Note roughly $f(X,Y)=\begin{pmatrix} k_1A-(k_2B+k_4+k_3)X+k_3XY \\ (k_2B-k_3)X-k_3XY \end{pmatrix}$

**Forward difference approximation**

$\lim_{\Delta t\to 0}\dfrac{f(X+\Delta t,Y)-f(X,Y)}{\Delta t}=\begin{pmatrix} -(k_2B+k_4+k_3) + k_3Y \\ (k_2B-k3) - k_3Y \end{pmatrix}$

$\lim_{\Delta t\to 0}\dfrac{f(X, Y+\Delta t)-f(X,Y)}{\Delta t}=\begin{pmatrix} k_3X \\ -k_3X \end{pmatrix}$ 

**Reaction-Diffusion system form**

Model the concentration $X(x,t)$ and $Y(x,t)$ with diffusion coefficients $D_X$ and $D_Y$ over a 1D spatial domain $x\in [0,L]$ governed by
$$\dfrac{\partial X}{\partial t}=D_X\dfrac{\partial ^2X}{\partial x^2}+k_1A-(k_2B+k_4+k_3)X+k_3XY$$
$$\dfrac{\partial Y}{\partial t}=D_Y\dfrac{\partial ^2Y}{\partial x^2}+(k_2B-k_3)X-k_3XY$$

**Boundary conditions**
1. Neumann (no-flux)
$$\dfrac{\partial X}{\partial x}|_{x=0}=0, \ \dfrac{\partial X}{\partial x}|_{x=L}=0$$
$$\dfrac{\partial Y}{\partial x}|_{x=0}=0, \ \dfrac{\partial Y}{\partial x}|_{x=L}=0$$
2. Dirichlet (fixed concentration)
$$X(0,t)=X_0, \ X(L,t)=X_L$$
$$Y(0,t)=Y_0, Y(L,t)=Y_L$$

In [ ]:
# Trials for the different possible combination of boundary conditions

# Oregonator